from category_encoders import TargetEncoder

def target_encode(train, test, columns, target):
    # Ensure the target column is passed as a Series or NumPy array
    if isinstance(target, pd.Series) or isinstance(target, np.ndarray):
        encoder = TargetEncoder(cols=columns)
        
        # Fit and transform train set
        train[columns] = encoder.fit_transform(train[columns], target)
        
        # Transform test set
        test[columns] = encoder.transform(test[columns])
        
        return train, test
    else:
        raise ValueError("Target must be a Series or an array.")

# Example of use
X_train, X_test = target_encode(X_train, X_test, ['contact_type', 'notify'], y_train)



def binary_encode(df, columns):
    encoder = BinaryEncoder(cols=columns)
    df = encoder.fit_transform(df)
    return df

X_train = binary_encode(X_train, ['contact_type'])
X_test = binary_encode(X_test, ['contact_type'])


def frequency_encode(df, columns):
    for column in columns:
        freq = df[column].value_counts(normalize=True)
        df[column] = df[column].map(freq)
    return df

X_train = frequency_encode(X_train, [ 'notify'])
X_test = frequency_encode(X_test, [ 'notify'])


def apply_get_dummies(train, test, columns):
    dummies_train = pd.get_dummies(train[columns], drop_first=True, dtype=int)
    dummies_test = pd.get_dummies(test[columns], drop_first=True, dtype=int)
    
    # Align columns of train and test to avoid mismatch
    dummies_train, dummies_test = dummies_train.align(dummies_test, join='outer', axis=1, fill_value=0)
    
    train = pd.concat([train.drop(columns, axis=1), dummies_train], axis=1)
    test = pd.concat([test.drop(columns, axis=1), dummies_test], axis=1)
    
    return train, test

X_train, X_test = apply_get_dummies(X_train, X_test, ['contact_type', 'notify'])
